In [1]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass(
        prompt="Enter your OpenAI API key (required if using OpenAI): "
    )

base_url = os.environ.get("BASE_URL")
model_name = os.environ.get("MODEL_NAME")
deepseek_model_name = os.environ.get("DEEPSEEK_MODEL_NAME")

print(base_url, model_name)

https://dashscope.aliyuncs.com/compatible-mode/v1 qwen-plus


In [4]:
from langchain_deepseek import ChatDeepSeek


model = ChatDeepSeek(
    model=deepseek_model_name,
    temperature=0,
    max_retries=2,
    base_url=base_url,
)

In [5]:
from pydantic import BaseModel, Field, model_validator
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup")
        if setup and setup[-1] != "?":
            raise ValueError("Badly formed question!")
        return values


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

# 使用链式

In [6]:
chain = prompt | model | parser
chain.invoke({"query": "Tell me a joke."})

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

## 使用 SimpleJsonOutputParser

In [7]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

In [8]:
list(json_chain.stream({"question": "Who invented the microscope?"}))

[{},
 {'answer': ''},
 {'answer': 'The'},
 {'answer': 'The microscope'},
 {'answer': 'The microscope was'},
 {'answer': 'The microscope was invented'},
 {'answer': 'The microscope was invented by'},
 {'answer': 'The microscope was invented by Zach'},
 {'answer': 'The microscope was invented by Zacharias'},
 {'answer': 'The microscope was invented by Zacharias Jans'},
 {'answer': 'The microscope was invented by Zacharias Janssen'},
 {'answer': 'The microscope was invented by Zacharias Janssen,'},
 {'answer': 'The microscope was invented by Zacharias Janssen, a'},
 {'answer': 'The microscope was invented by Zacharias Janssen, a Dutch'},
 {'answer': 'The microscope was invented by Zacharias Janssen, a Dutch spectacle'},
 {'answer': 'The microscope was invented by Zacharias Janssen, a Dutch spectacle maker'},
 {'answer': 'The microscope was invented by Zacharias Janssen, a Dutch spectacle maker,'},
 {'answer': 'The microscope was invented by Zacharias Janssen, a Dutch spectacle maker, arou

In [9]:
list(chain.stream({"query": "Tell me a joke."}))



[Joke(setup="Why don't scientists trust atoms?", punchline=''),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they make'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything'),
 Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')]